In [ ]:
data_paths = {
    'Micro': "../data/runs/2018-11-10-micro-rad-khype1e6/",
    'NNOrig': "../data/runs/2018-11-09-model188-equilibriation-penalty",
    'NN': "../data/runs/2018-11-10-model188-khyp1e6-rerun/"
}

ng_path = "../data/processed/training.nc"

In [ ]:
from toolz import valmap        
from src.data.sam import SAMRun
from uwnet.thermo import compute_q2
import xarray as xr
import holoviews as hv
hv.extension('bokeh')
%opts Image[width=600, height=400, colorbar=True](cmap='viridis')
%opts Curve[width=400]

def plot_xy(da, dynamic=True):
    return hv.Dataset(da).to.image(["x", "y"], dynamic=dynamic)

In [ ]:
data_paths = {
    'Micro': "../data/runs/2018-11-10-micro-rad-khype1e6/",
    'NNOrig': "../data/runs/2018-11-09-model188-equilibriation-penalty",
    'NN': "../data/runs/2018-11-10-model188-khyp1e6-rerun/"
}

ng_path = "../data/processed/training.nc"
ngaqua =  xr.open_dataset(ng_path).isel(step=0)

In [ ]:
runs = valmap(SAMRun, data_paths)
ngaqua =  xr.open_dataset(ng_path).isel(step=0)

# Net Precip

## Spinup

In [ ]:
def get_first_steps(ds):
    first_step = ds.isel(time=slice(1,4))
    first_step['time'] = (first_step.time-first_step.time[0])*86400
    first_step.time.attrs['units'] = 's'
    return first_step

ds = runs['NN'].data_3d
first_step = get_first_steps(ds)
first_step.FQTNN.mean(['x']).plot(col='time')

Here is a comparision of P-E over these first few time steps:

In [ ]:
net_precip_nn = -(ngaqua.layer_mass * ds.FQTNN).sum('z')/1000
net_precip_nn['time'] = (net_precip_nn.time -net_precip_nn.time[0])*24
net_precip_nn[[1, 5, 6, 7]].mean('x').plot(hue='time')
plt.title("Zonal mean P-E (time in hours)");

## Comparison to NG-Aqua

Let's open the NGAqua data and plot Q2

In [ ]:
from uwnet.thermo import compute_q2

q2 = compute_q2(ngaqua.isel(time=slice(0, 2))).dropna('time')

Here is thea actual mean Q2 in the first tiem step

In [ ]:
q2.mean('x').plot()

Here is a comparision of the zonally averaged net precipitation.

In [ ]:
q2_int = -(ngaqua.layer_mass * q2).sum('z')/1000
fqtnn_int = -(ngaqua.layer_mass * first_step.FQTNN).sum('z')/1000

q2_int.mean(['x']).plot(label='NGaqua')
fqtnn_int[0].mean('x').plot(label='NN-prediction')
plt.title("Zonal mean of Net Precip")
plt.legend()

There is some systematic difference, but overally it is too noisy to tell from the first time step

# Spin up of vertical velocity

In [ ]:
plot_data = runs['NN'].data_3d.W[4::3, 8]
dmap = plot_xy(plot_data, dynamic=False)\
           .relabel(f"W at z={float(plot_data.z)}")
dmap